# NamedFunction helper (SymPy) — a hands-on notebook

This notebook demonstrates the **`NamedFunction` helper** (exported by `gu_toolkit`) for creating **custom SymPy functions** with:

- **Readable documentation** (auto-generated “NamedFunction notes”)
- **Correct `inspect.signature(...)`** on the generated SymPy Function class
- A **`rewrite("expand_definition")`** path that reveals the symbolic definition
- Optional **vectorized NumPy evaluation** via `f_numpy` (function-mode) or `numeric()` (class-mode)

We’ll also build a few “enriched” symbolic functions (e.g. *vanishes on integers*, *unit-modulus complex map*) and use standard SymPy transformations (differentiation, series, simplification, substitution, solving).

> **Conventions:** After `gu_toolkit.setup()`, SymPy names are injected into the global namespace, so we will **not** use `import sympy as sp` and we will **not** write `sympy.sin`, etc.


## 0. Setup

In [1]:
import gu_toolkit
gu_toolkit.setup()   # injects SymPy + plugin exports into notebook namespace

🔧 Initializing GU Toolkit...
✓ Matplotlib backend set to 'widget'
[gu_toolkit] Skip (disabled) gu_toolkit.plugins.example
[gu_toolkit] Loaded gu_toolkit.plugins.NamedFunction (exports=1, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartException.SmartException (exports=1, hook=True)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartFigure.SmartFigure (exports=9, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.SmartParameters.SmartParameters (exports=4, hook=False)
[gu_toolkit] Loaded gu_toolkit.plugins.numpify (exports=1, hook=False)
✅ Smart Exception Handler Activated.
✅ Smart Exception Handler Activated.
[gu_toolkit] Exported 950 names into the notebook namespace.
🎓 GU Toolkit Ready.


## 1. Sanity checks

Let’s confirm that the expected objects are available in the global namespace after `setup()`.

- `NamedFunction` should be present.
- Basic SymPy constructors like `Symbol`, `symbols`, and functions like `sin` should also be present.


In [2]:
import inspect

# Expect these names in globals after gu_toolkit.setup()
print("NamedFunction:", NamedFunction)
print("Symbol:", Symbol)
print("sin:", sin)

# The helper produces SymPy Function *classes*; you call them like functions.
print("NamedFunction is callable:", callable(NamedFunction))

NamedFunction: <function NamedFunction at 0x00000264538F45E0>
Symbol: <class 'sympy.core.symbol.Symbol'>
sin: sin
NamedFunction is callable: True


## 2. Quick start: function-mode decorator

### What function-mode does

When you write

```python
@NamedFunction
def F(x):
    return x**2 + 1
```

you get a **new SymPy `Function` subclass** called `F`. It *does not expand automatically* in expressions, but it supports:

- `expr.rewrite("expand_definition")` → returns the underlying expression
- `expr.evalf()` → evaluates by rewriting first (when rewrite succeeds)
- A rich docstring that records the current definition (best-effort)

Let’s build a minimal example and manipulate it symbolically.


In [3]:
@NamedFunction
def SquarePlusOne(x):
    """Return x^2 + 1 as a NamedFunction example."""
    return x**2 + 1

x = Symbol("x")

expr = SquarePlusOne(x)
expr

SquarePlusOne(x)

### 2.1 Expanding (rewriting) the definition

In [4]:
expr_expanded = expr.rewrite("expand_definition")
expr_expanded

x**2 + 1

### 2.2 Calculus and algebra: differentiate, integrate, simplify

In [7]:
dexpr = diff(expr, x)               # derivative of an *unevaluated* named function
dexpr_rewritten = dexpr.rewrite("expand_definition")

iexpr = integrate(expr, x)            # integral of unevaluated named function (formal)
iexpr_rewritten = iexpr.rewrite("expand_definition")

for e in (expr, dexpr, dexpr_rewritten, iexpr, iexpr_rewritten):
    display(e)

SquarePlusOne(x)

Derivative(SquarePlusOne(x), x)

Derivative(x**2 + 1, x)

Integral(SquarePlusOne(x), x)

Integral(x**2 + 1, x)

**Note.** If you want the calculus to “see” the definition, rewrite first:

```python
diff(expr.rewrite("expand_definition"), x)
```

That is a general pattern with NamedFunction: treat it as an *opaque symbol* until you opt in to expansion.


In [9]:
diff(expr.rewrite("expand_definition"), x)

2*x

The difference is that in the first example we first computed the derivative, then expanded the defintion. Instead, in the second example, we expand the definition and then operato on the resulting expression.

To actually perform the operations, we can use `doit()`

In [10]:
for e in (expr, dexpr, dexpr_rewritten, iexpr, iexpr_rewritten):
    display(e.doit())

SquarePlusOne(x)

Derivative(SquarePlusOne(x), x)

2*x

Integral(SquarePlusOne(x), x)

x**3/3 + x

### 2.3 Signature and docstring quality

In [12]:
#Print documentation:
SquarePlusOne?

Init signature: SquarePlusOne(x)
Docstring:     
Return x^2 + 1 as a NamedFunction example.

## NamedFunction notes


-----

- **Numerical Implementation**:

    ABSENT. No custom implementation provided; falls back to SymPy's `lambdify`.
- **Symbolic Expansion**:

    To view the underlying symbolic definition programmatically, apply the rewrite method:
    ```python
    expr.rewrite("expand_definition")
    ```
- **Current Definition**:

    `SquarePlusOne(x) = x**2 + 1`

    $ \mathrm{SquarePlusOne}(x) = \mathtt{\text{x**2 + 1}} $
Type:           _SignedFunctionMeta
Subclasses:     

In [11]:
print("inspect.signature(SquarePlusOne):", inspect.signature(SquarePlusOne))
print()
print("Docstring excerpt:")
print("\n".join(SquarePlusOne.__doc__.splitlines()[:20]))

inspect.signature(SquarePlusOne): (x)

Docstring excerpt:
Return x^2 + 1 as a NamedFunction example.

## NamedFunction notes


-----

- **Numerical Implementation**:

    ABSENT. No custom implementation provided; falls back to SymPy's `lambdify`.
- **Symbolic Expansion**:

    To view the underlying symbolic definition programmatically, apply the rewrite method:
    ```python
    expr.rewrite("expand_definition")
    ```
- **Current Definition**:

    `SquarePlusOne(x) = x**2 + 1`



## 3. Opaque functions (return `None`)

Sometimes you want a symbolic function placeholder with a name (e.g. “unknown transfer function”),
but you *don’t* want an explicit symbolic definition.

In **function-mode**, returning `None` tells `NamedFunction` that the function is **opaque**:

- `rewrite("expand_definition")` does nothing (returns the function call)
- `evalf()` will not produce a numeric value via rewriting
- you can still differentiate formally (`Derivative(Mystery(x), x)`)

This is useful for building expressions while postponing definitions.


In [15]:
@NamedFunction
def Mystery(x):
    """An intentionally opaque function: it has no explicit symbolic definition."""
    return None

expr2 = Mystery(x) + 3*SquarePlusOne(x)
for e in (expr2, expr2.rewrite("expand_definition")):
    display(e)

Mystery(x) + 3*SquarePlusOne(x)

3*x**2 + Mystery(x) + 3

In [14]:
diff(Mystery(x), x)

Derivative(Mystery(x), x)

## 4. Custom NumPy evaluation in function-mode (`f_numpy`)

The helper supports an optional **vectorized NumPy** implementation by attaching a function attribute
named `f_numpy` **before** decoration.

Because decorators run immediately, the easiest pattern in a notebook is:

```python
@NamedFunction
def F(x): ...

def F_numpy(x): ...
F.f_numpy = F_numpy
```

This is especially helpful when your symbolic expression is complicated or when you want tight control
over numeric behavior.


In [16]:
import numpy as np
@NamedFunction
def SmoothStep(x):
    """A smooth step: (1 + tanh(x)) / 2."""
    return (1 + tanh(x)) / 2

def SmoothStep_numpy(x):
    x = np.asarray(x, dtype=float)
    return (1 + np.tanh(x)) / 2

# Attach numeric implementation to the wrapper
SmoothStep.f_numpy = SmoothStep_numpy


print("Has f_numpy:", hasattr(SmoothStep, "f_numpy") and SmoothStep.f_numpy is not None)
print("Signature:", inspect.signature(SmoothStep))

Has f_numpy: True
Signature: (x)


### 4.1 Using the symbolic definition (via rewrite)

In [18]:
expr3 = SmoothStep(x)
for e in (expr3, expr3.rewrite("expand_definition")):
    display(e)

SmoothStep(x)

tanh(x)/2 + 1/2

### 4.2 Calling `f_numpy` directly (vectorized)

In [22]:
import pandas as pd

xs = np.linspace(-3, 3, 7)
ys = SmoothStep.f_numpy(xs)
display(
    pd.DataFrame({'x': xs, 'y': ys}).style.hide()
)


x,y
-3.000000,0.002473
-2.000000,0.017986
-1.000000,0.119203
0.000000,0.500000
1.000000,0.880797
2.000000,0.982014
3.000000,0.997527


## 5. Class-mode decorator (symbolic + numeric)

### What class-mode does

Class-mode is an alternative syntax for defining a `NamedFunction`. It is designed for the case where you want:

- a **symbolic** definition in `symbolic(self, *args)` (may return a SymPy expression or `None`)
- a **vectorized numeric** implementation in `numeric(self, *args)`

You write:

```python
@NamedFunction
class MyFunc:
    def symbolic(self, x): ...
    def numeric(self, x): ...
```

The resulting SymPy function class is named after your class (here `MyFunc`).


### 5.1 Example: a robust `Sinc(x) = sin(x)/x` with a removable singularity at 0

In [23]:
@NamedFunction
class Sinc:
    """Sinc(x) = sin(x)/x, with the value at x=0 defined as 1."""
    def symbolic(self, x):
        return Piecewise((1, Eq(x, 0)), (sin(x)/x, True))

    def numeric(self, x):
        x = np.asarray(x, dtype=float)
        y = np.empty_like(x)
        mask0 = (x == 0.0)
        y[mask0] = 1.0
        y[~mask0] = np.sin(x[~mask0]) / x[~mask0]
        return y

print("Signature:", inspect.signature(Sinc))
print("Has f_numpy:", hasattr(Sinc, "f_numpy") and Sinc.f_numpy is not None)

Signature: (x)
Has f_numpy: True


In [24]:
display(Sinc(x))
display(Sinc(x).rewrite("expand_definition"))

Sinc(x)

Piecewise((1, Eq(x, 0)), (sin(x)/x, True))

### 5.2 Symbolic expansion and numerical evaluation

In [27]:
# evalf() rewrites first, then evalf()s the expression
expr_sinc.subs(x, 0).evalf(), expr_sinc.subs(x, 0.3).evalf()

(1.00000000000000, 0.985067355537799)

In [28]:
# Vectorized numeric evaluation
xs = np.array([0.0, 0.1, 0.5, 1.0])
Sinc.f_numpy(xs)

array([1.        , 0.99833417, 0.95885108, 0.84147098])

## 6. LaTeX-aware naming and argument labels

The helper tries to build nicer **docstring** representations by:

- preserving standard Greek names (alpha, beta, …)
- wrapping multi-letter heads in `\mathrm{...}`
- handling simple subscripts (like `x_val` → `x_{val}`)

This affects the auto-generated docstring notes, and it can help with readability.


In [29]:
@NamedFunction
def energy_density(x_val, y_val):
    """A toy example using subscripted parameter names."""
    return x_val**2 + y_val**2

x_val, y_val = symbols("x_val y_val")
expr_ed = energy_density(x_val, y_val)
expr_ed

energy_density(x_val, y_val)

In [31]:
energy_density?

Init signature: energy_density(x_val, y_val)
Docstring:     
A toy example using subscripted parameter names.

## NamedFunction notes


-----

- **Numerical Implementation**:

    ABSENT. No custom implementation provided; falls back to SymPy's `lambdify`.
- **Symbolic Expansion**:

    To view the underlying symbolic definition programmatically, apply the rewrite method:
    ```python
    expr.rewrite("expand_definition")
    ```
- **Current Definition**:

    `energy_density(x_val, y_val) = x_val**2 + y_val**2`

    $ \mathrm{energy_density}(x_{val}, y_{val}) = \mathtt{\text{x\_val**2 + y\_val**2}} $
Type:           _SignedFunctionMeta
Subclasses:     

In [30]:
print("\n".join(energy_density.__doc__.splitlines()[-20:]))


## NamedFunction notes


-----

- **Numerical Implementation**:

    ABSENT. No custom implementation provided; falls back to SymPy's `lambdify`.
- **Symbolic Expansion**:

    To view the underlying symbolic definition programmatically, apply the rewrite method:
    ```python
    expr.rewrite("expand_definition")
    ```
- **Current Definition**:

    `energy_density(x_val, y_val) = x_val**2 + y_val**2`

    $ \mathrm{energy_density}(x_{val}, y_{val}) = \mathtt{\text{x\_val**2 + y\_val**2}} $


## 7. “Enriched” symbolic functions with custom logic

NamedFunction’s rewrite hook simply calls your `symbolic` definition (function-mode or class-mode).
That means you can embed **custom symbolic logic** in the definition itself.

Below are two patterns:

1. A function that **vanishes on integers** (and otherwise behaves like a nice analytic expression).
2. A function of a complex variable with **absolute value 1** away from 0.


### 7.1 Vanishes on integers

In [40]:
@NamedFunction
def VanishOnIntegers(t):
    """A function that returns 0 on integer inputs; otherwise behaves like sin(pi t)."""
    # If SymPy knows t is an integer (e.g., Integer(3) or Symbol(..., integer=True)), return 0
    if t.is_integer is True: # if t is KNOWN to be an integer
        return Integer(0) 
    if t.is_integer is False: # if t is KNOWN not to be an integer
        return sin(pi*t)    
    if t.is_integer is None: # if t is UNKNOWN
        return None # This should actually be the default so let us return it also outside the if
    return None 
t = Symbol("t")
k = Symbol("k", integer=True)

expr_v = VanishOnIntegers(t)
for e in (expr_v, expr_v.rewrite("expand_definition")):
    display(e)

VanishOnIntegers(t)

VanishOnIntegers(t)

In [43]:
# Concrete integer evaluation
for e in (VanishOnIntegers(3).rewrite("expand_definition"), VanishOnIntegers(3)):
    display(e)

0

VanishOnIntegers(3)

In [36]:
# Symbol known to be integer via assumptions
VanishOnIntegers(k).rewrite("expand_definition")

0

**Tip.** The check `t.is_integer is True` is deliberate. For a generic symbol `t`, `t.is_integer`
may be `None` (unknown), so the definition should fall back to a general expression.


### 7.2 A unit-modulus map for complex numbers (|u(z)| = 1 for z ≠ 0)

In [46]:
@NamedFunction
def Unimodular(z):
    """Return z/Abs(z) for z≠0 and 0 at z=0. (|z/Abs(z)|=1 away from 0)"""
    return Piecewise((0, Eq(z, 0)), (z/Abs(z), True))

z = Symbol("z")
z_nz = Symbol("z_nz", nonzero=True)

u = Unimodular(z)
for e in (u, u.rewrite("expand_definition")):
    display(e)

Unimodular(z)

Piecewise((0, Eq(z, 0)), (z/Abs(z), True))

In [47]:
# Absolute value: as an identity, Abs(z/Abs(z)) doesn't always simplify without assumptions.
Abs(Unimodular(z).rewrite("expand_definition"))

Abs(Piecewise((0, Eq(z, 0)), (z/Abs(z), True)))

In [48]:
# With a nonzero assumption, simplification is stronger:
Abs(Unimodular(z_nz).rewrite("expand_definition")).simplify()

Abs(z_nz/Abs(z_nz))

## 8. Symbolic manipulation “mini-gallery”

This section shows a few standard SymPy workflows involving NamedFunctions:

- substitution and composition
- series expansions (rewrite first)
- solving equations
- controlling evaluation (keeping or expanding the definition)


### 8.1 Composition and substitution

In [ ]:
expr_comp = SquarePlusOne(Sinc(x)) + SmoothStep(x)
expr_comp

In [ ]:
expr_comp.rewrite("expand_definition")

### 8.2 Series expansion (rewrite first)

In [ ]:
series(expr_sinc.rewrite("expand_definition"), x, 0, 6)

In [ ]:
series(SmoothStep(x).rewrite("expand_definition"), x, 0, 6)

### 8.3 Solving an equation involving a NamedFunction

In [ ]:
sol = solve(Eq(SquarePlusOne(x).rewrite("expand_definition"), 5), x)
sol

### 8.4 Mixing opaque and defined functions

In [ ]:
expr_mix = Mystery(x) + SquarePlusOne(x) + VanishOnIntegers(x)
expr_mix

In [ ]:
expr_mix.rewrite("expand_definition")

## 9. Practical tips and gotchas

- **Nothing expands automatically.** Use `rewrite("expand_definition")` when you want the actual definition.
- **Opaque functions** (`return None`) are useful placeholders; they stay unevaluated.
- **Custom NumPy (`f_numpy`) in function-mode** must be attached *before* wrapping the function.
- **Class-mode numeric** is always present, exposed as `MyFunc.f_numpy` on the resulting SymPy function class.
- If you want deeper SymPy integration (e.g. custom evaluation rules, simplification hooks, assumptions),
  you may still write a custom `sympy.Function` subclass. NamedFunction is meant to cover the common “named wrapper” case.

### Optional: introspection helpers


In [ ]:
# Introspect where NamedFunction came from (useful when debugging plugin loading)
print("NamedFunction module:", NamedFunction.__module__)
print("NamedFunction object:", NamedFunction)